In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]

In [6]:
train_data = pd.read_csv("/content/drive/MyDrive/vgg16/Dataset/KDDTrain+.txt", names=feature)
test_data = pd.read_csv("/content/drive/MyDrive/vgg16/Dataset/KDDTest+.txt", names=feature)
test_data21 = pd.read_csv("/content/drive/MyDrive/vgg16/Dataset/KDDTest-21.txt", names=feature)

data = pd.concat([train_data, test_data], ignore_index=True)

In [7]:
data.drop(["difficulty"], axis=1, inplace=True)

In [8]:
def label_change(df):
    df.label.replace(['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm'],'Dos',inplace=True)
    df.label.replace(['ftp_write','guess_passwd','imap','multihop','phf','spy','warezclient','warezmaster','snmpgetattack','httptunnel','snmpguess','sendmail','named','xlock','xsnoop'],'R2L',inplace=True)
    df.label.replace(['ipsweep','nmap','portsweep','satan','mscan','saint'],'Probe',inplace=True)
    df.label.replace(['buffer_overflow','loadmodule','perl','rootkit','ps','xterm','sqlattack'],'U2R',inplace=True)

In [9]:
label_change(data)

In [10]:
label = pd.DataFrame(data.label)

In [11]:
scaler = StandardScaler()

def standardization(data, col):
    for i in col:
        arr = data[i]
        arr = np.array(arr)
        data[i] = scaler.fit_transform(arr.reshape(len(arr), 1))
    return data

numeric_col = data.select_dtypes(include='number').columns
data = standardization(data, numeric_col)

In [12]:
le = preprocessing.LabelEncoder()
encode = label.apply(le.fit_transform)
data['intrusion'] = encode

In [13]:
data.drop(['label'], axis=1, inplace=True)

In [14]:
data = pd.get_dummies(data, columns=['protocol_type','service','flag'], prefix="", prefix_sep="")

In [ ]:
y_data = data['intrusion']
X_data = data.drop(labels=['intrusion'], axis=1)

print('X_train has shape:', X_data.shape, '\nY_train has shape:', y_data.shape)

In [16]:
num_classes = len(np.unique(y_data))

y_data = LabelBinarizer().fit_transform(y_data)
X_data = np.array(X_data)
y_data = np.array(y_data)

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
print("Data preprocessing completed.")

Data preprocessing completed.


In [35]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [30]:
model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Adjusted for the number of classes

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: In

In [ ]:
# Evaluate on test data
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test Accuracy:", accuracy)

In [ ]:
# Get predictions
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)
Y_test_labels = np.argmax(Y_test, axis=1)

In [ ]:
# Confusion matrix and classification report
conf_matrix = confusion_matrix(Y_test_labels, y_pred)
class_report = classification_report(Y_test_labels, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)